In [57]:
%pip install pandas chromadb openai tiktoken 
import openai
import chromadb
import pandas as pd
from chromadb.utils import embedding_functions
from tiktoken import encoding_for_model
import os

Note: you may need to restart the kernel to use updated packages.


In [58]:
from dotenv import load_dotenv
import os

#  load environment variables from .env file (this is for security reasons)
load_dotenv()

# get API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

In [75]:
import time
# define embedding function using OpenAI text-embedding-ada-002 model
def get_embeddings(texts):
    try:
        response = openai.embeddings.create(
            model='text-embedding-ada-002',
            input=texts
        )
        return [item['embedding'] for item in response['data']]
    except openai.error.RateLimitError:
        print("Rate limit exceeded. Waiting for 60 seconds before retrying...")
        time.sleep(60)
        return get_embeddings(texts)

In [76]:
client = chromadb.Client()
# get 'text_embeddings' collection. if you don't have a collection created yet, create it
collection_name = 'text_embeddings'
if collection_name in [col.name for col in client.list_collections()]:
    collection = client.get_collection(collection_name)
else:
    collection = client.create_collection(collection_name)

###
load all the files we want to do vector embeddings for

In [77]:
organized_blow_molding_basics_csv = r'organized_blow_molding_basics.csv'
organized_blow_molding_basics_df = pd.read_csv(organized_blow_molding_basics_csv)

organized_blow_molding_basics_df.head()


,section,text
0,Introduction,Blow molding is defined as a plastic process w...
1,Resins,Most thermoplastic resins in use in the plasti...
2,Processes,There are three main processes used by the blo...
3,Injection Blow Moulding,"In injection blow molding, melted plastic resi..."
4,Injection Blow-Mold Tooling,Injection blow-molding requires two molds: one...


In [78]:
organized_pet_bottle_faults_csv = r'organized_pet_bottle_faults.csv'
organized_pet_bottle_faults_df = pd.read_csv(organized_pet_bottle_faults_csv)

organized_pet_bottle_faults_df.head()


,contents,description,cause,solution
0,BLOWN FINISH,"Expansion of the finish, located at or above t...",1. Oven ambient temperature too high.\n2. Stan...,1. Increase oven ventilation or direct air on ...
1,SEALING SURFACE AND NECK SUPPORT RING DAMAGE,As the transfer arm releases the preform into ...,1. Poor transfer arm positioning causing misal...,1. Re-align transfer arm — height and timing.\...
2,NECK CONSTRICTION,A fold of material in the neck or upper should...,"1. Low pre-blow, delayed or missing.\n2. Flow ...","1. If problem is occurring on all stations, mo..."
3,PERPENDICULARITY,The distortion may occur at or above the neck ...,1. Mis-alignment of the preform to the mould o...,1. Re-align transfer arm (height and timing). ...
4,HARD SHOULDER,An excess of material remains in the neck and ...,1. Incorrect heat profile.\n2. Incorrect blowi...,1. Reduce heat in the body and/or base before ...


In [79]:
organized_stressesinpebottles_csv = r'organized_stressesinpebottles.csv'
organized_stressesinpebottles_df = pd.read_csv(organized_stressesinpebottles_csv)

organized_stressesinpebottles_df.head()


,section,text
0,ABSTRACT,"A commercial blow-molding grade, high-density ..."
1,INTRODUCTION,Commercial bottles must satisfy performance cr...
2,EXPERIMENTAL,"A commercial blow-molding grade, high-density\..."
3,RESULTS AND DISCUSSION,A typical three-dimensional thickness distribu...
4,CONCLUSIONS,"The results of this study suggest that, in the..."


In [80]:
dupont_table_csv = r'Dupont_Table7.csv'
dupont_table_df = pd.read_csv(dupont_table_csv)

dupont_table_df.head()


,Fault,Cause,Solution
0,Bubbles in melt,Moisture,Dry resin
1,Bubbles in melt,Wrong screw design (air entrapment),Fit suitable screw
2,Poor melt strength,Moisture,Dry resin
3,Poor melt strength,Melt temperature too high,Check with needle pyrometer and\ncorrect to re...
4,Excessive melt temperature (significantly abov...,Faulty temperature controllers or thermocouples,Repair/calibrate controllers and thermocouples


In [81]:
#add all the csvs into a list 
dataframes = [organized_blow_molding_basics_df, organized_stressesinpebottles_df]

In [82]:

# loop through the dataframe and add embeddings to ChromaDB
for df in dataframes:
    for idx, row in df.iterrows():
        text = row['text']  # Assuming the text you want to embed is in the 'text' column
        embedding = get_embedding(text)
        
        # Add the embedding to the ChromaDB collection
        collection.add(
            embeddings=[embedding], 
            metadatas=[{'text': text, 'doc_id': idx}],  # Metadata for future reference
            ids=[str(idx)]  # Unique ID for each row
        )

print("Data added to ChromaDB!")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}